In [ ]:
# import reddit api wrapper
import praw
import pandas as pd
import numpy as np
import datetime
import sqlite3

from textblob import TextBlob

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

In [ ]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

In [ ]:
# access password and client secret id via local files
with open('pw.txt', 'r') as file1:
    pw = file1.read()

with open('client_secret.txt', 'r') as file2:
    cs = file2.read()

# create a praw Reddit instance with app credentials and secret info passed through
reddit = praw.Reddit(
    client_id="XbesrQBvKymjgLdgg_D6lA",
    client_secret=cs,
    user_agent="NFLTextAnalysis/0.0.1",
    username="ta_api",
    password=pw
)

In [ ]:
# read in the gamethreads ids from sql
gamethreads = pd.read_csv('Data/gamethreads_2021.csv')

In [ ]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all submission ids that have previously been crawled
    df = pd.read_sql_query("SELECT * FROM comments", conn)
    crawled_ids = df['submission_id'].unique().tolist()

    all_ids = gamethreads['submission_id'].unique().tolist()

    # yields the elements in all_ids not in crawled_ids
    remaining_ids = np.setdiff1d(all_ids, crawled_ids)
    print("Threads remaining: " + str(len(remaining_ids)))

    for each_id in remaining_ids:

        comment_df = nfl_gamethreads.get_comments(reddit, each_id)
        
        comment_df.to_sql(name='comments', con=conn, if_exists='append', index=False)

